In [1]:
import sounddevice as sd

# 입력 장치 목록을 가져오는 함수
def show_input_devices():
    devices = sd.query_devices()
    for idx, device in enumerate(devices):
        if device['max_input_channels'] > 0:  # 입력 채널이 있는 장치만 출력
            print(f"Device ID: {idx}, Name: {device['name']}")

# 입력 장치 목록 출력
show_input_devices()


Device ID: 0, Name: Microsoft Sound Mapper - Input
Device ID: 1, Name: 마이크(4- MATA STUDIO C10)
Device ID: 6, Name: 주 사운드 캡처 드라이버
Device ID: 7, Name: 마이크(4- MATA STUDIO C10)
Device ID: 15, Name: 마이크(4- MATA STUDIO C10)
Device ID: 18, Name: 마이크 (MATA STUDIO C10)


In [2]:
import sounddevice as sd
import numpy as np
import librosa
import tensorflow as tf
import time
from PyQt5 import QtCore, QtGui, QtWidgets
from PyQt5.QtWidgets import QApplication, QWidget
from PyQt5.QtGui import QIcon
import webrtcvad
import pyaudio
import wave
import noisereduce as nr
from faster_whisper import WhisperModel
import datetime
import os
import sys

model_path = os.path.join(os.getcwd(), 'my_model.h5')
model = tf.keras.models.load_model(model_path)

class_labels = ['air_conditioner', 'car_horn', 'children_playing', 'dog_bark', 'drilling',
                 'engine_idling', 'gun_shot', 'jackhammer', 'siren', 'street_music']



def preprocess_audio(audio, sample_rate=22050):
    mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)

   
    if mfccs.shape[1] < 174: 
        mfccs = np.pad(mfccs, ((0, 0), (0, 174 - mfccs.shape[1])), mode='constant')
    
    return mfccs

class STTWorker(QtCore.QThread):
    text_update = QtCore.pyqtSignal(str)  
    stt_finished = QtCore.pyqtSignal(str) 

    def __init__(self, model_size="small"):
        super(STTWorker, self).__init__()
        self.vad = webrtcvad.Vad(3)
        self.model = WhisperModel(model_size, device="cpu")
        self.sample_rate = 16000
        self.frame_duration = 20
        self.frame_size = int(self.sample_rate * self.frame_duration / 1000)
        self.channels = 1
        self.running = False
        self.frames = []

        self.p = pyaudio.PyAudio()
        self.stream = self.p.open(format=pyaudio.paInt16, channels=self.channels,
                                  rate=self.sample_rate, input=True,
                                  frames_per_buffer=self.frame_size)

    def run(self):
        self.running = True
        self.frames = []
        while self.running:
            audio_frame = self.stream.read(self.frame_size)
            self.frames.append(audio_frame)

    def stop(self):
        self.running = False
        self.process_audio()

    def process_audio(self):
        audio_data = b''.join(self.frames)
        clean_segment = self.reduce_noise(audio_data)
        stt_result = self.transcribe_audio(clean_segment)
        for text_segment in stt_result:
            self.stt_finished.emit(text_segment.text)

    def reduce_noise(self, audio_data):
        audio_array = np.frombuffer(audio_data, dtype=np.int16).astype(np.float32)
        reduced_noise = nr.reduce_noise(y=audio_array, sr=self.sample_rate)
        return reduced_noise.astype(np.int16).tobytes()

    def transcribe_audio(self, audio_data):
        audio_array = np.frombuffer(audio_data, dtype=np.int16).astype(np.float32) / 32768.0
        segments, _ = self.model.transcribe(audio_array, language='ko')
        return segments

class SoundClassifierWorker(QtCore.QThread):
    sound_classified = QtCore.pyqtSignal(str) 

    def __init__(self, parent=None):
        super(SoundClassifierWorker, self).__init__(parent)
        self.running = True  

    def run(self):
        while self.running:
            self.classify_sound()  
            time.sleep(2)  

    def classify_sound(self):
        sample_rate = 22050
        duration = 3 

        audio = sd.rec(int(sample_rate * duration), samplerate=sample_rate, channels=1)
        sd.wait()  

        audio = audio.flatten()  
        mfccs = preprocess_audio(audio, sample_rate)

        mfccs = np.expand_dims(mfccs, axis=-1)  
        mfccs = np.expand_dims(mfccs, axis=0)   

        prediction = model.predict(mfccs)
        prediction_percentage = prediction[0] * 100  

        current_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        
        result_text = ""
        highest_confidence = 0
        highest_label = ""

        # 제외할 레이블들의 인덱스
        excluded_indices = [0, 2, 5, 6, 9]  # air_conditioner, children_playing, engine_idling, gun_shot, street_music

        # 레이블과 예측 정확도를 저장할 리스트
        prediction_results = []

        # 제외된 레이블을 무시하고 결과 처리
        for i, label in enumerate(class_labels):
            if i not in excluded_indices:
                prediction_results.append((label, prediction_percentage[i]))

                if prediction_percentage[i] > highest_confidence:
                    highest_confidence = prediction_percentage[i]
                    highest_label = label

        # 정확도에 따라 내림차순으로 정렬
        prediction_results.sort(key=lambda x: x[1], reverse=True)

        # 콘솔 지우기 (화면 초기화)
        print("\033[H\033[J", end="")  # ANSI 코드로 화면 지우기

        # 포맷팅된 출력 만들기
        print("---------------------------------------------")
        for idx, (label, confidence) in enumerate(prediction_results, start=1):
            print(f"{idx}. {label}: {confidence:.2f}% ({current_time})")
        print("---------------------------------------------")

        # 가장 높은 확률이 60% 이하인 경우, 지정된 메시지 출력
        if highest_confidence <= 60:
            result_text = "No sound detected with more than 70% confidence."
        else:
            # 가장 높은 확률의 라벨 출력 (PyQt 창에 표시)
            result_text = f"가장 높은 소리: {highest_label} - {highest_confidence:.2f}%"
        
        # PyQt 창에 결과 전송
        self.sound_classified.emit(result_text)

    def stop(self):
        self.running = False  

class Ui_STT(object):
    def setupUi(self, STT):
        STT.setObjectName("STT")
        STT.resize(640, 480)  # Updated window size to 640x480
        STT.setCursor(QtGui.QCursor(QtCore.Qt.ArrowCursor))

        self.textEdit = QtWidgets.QTextEdit(STT)
        self.textEdit.setGeometry(QtCore.QRect(20, 20, 600, 300))  # Adjusted to fit within 640x480
        self.textEdit.setObjectName("textEdit")

        self.pushButton = QtWidgets.QPushButton(STT)
        self.pushButton.setGeometry(QtCore.QRect(20, 340, 101, 40))  # Adjusted button size and position
        self.pushButton.setObjectName("pushButton")

        self.stopButton = QtWidgets.QPushButton(STT)  
        self.stopButton.setGeometry(QtCore.QRect(20, 390, 101, 40))  
        self.stopButton.setObjectName("stopButton")

        self.pushButton_2 = QtWidgets.QPushButton(STT)
        self.pushButton_2.setGeometry(QtCore.QRect(140, 340, 101, 40))  # Adjusted button size and position
        self.pushButton_2.setObjectName("pushButton_2")

        self.pushButton_3 = QtWidgets.QPushButton(STT)
        self.pushButton_3.setGeometry(QtCore.QRect(260, 340, 101, 40))  # Adjusted button size and position
        self.pushButton_3.setObjectName("pushButton_3")

        self.pushButton_small = QtWidgets.QPushButton(STT)
        self.pushButton_small.setGeometry(QtCore.QRect(380, 340, 101, 40))  # Adjusted button size and position
        self.pushButton_small.setObjectName("pushButton_small")

        self.pushButton_medium = QtWidgets.QPushButton(STT)
        self.pushButton_medium.setGeometry(QtCore.QRect(500, 340, 101, 40))  # Adjusted button size and position
        self.pushButton_medium.setObjectName("pushButton_medium")

        self.newButton = QtWidgets.QPushButton(STT)  
        self.newButton.setGeometry(QtCore.QRect(500, 390, 101, 40))  # Adjusted button size and position
        self.newButton.setObjectName("newButton")

        self.retranslateUi(STT)
        QtCore.QMetaObject.connectSlotsByName(STT)

        self.pushButton.clicked.connect(self.start_recording)
        self.pushButton_2.clicked.connect(self.stop_recording)
        self.pushButton_3.clicked.connect(self.clear_text)
        self.pushButton_small.clicked.connect(self.set_small_model)
        self.pushButton_medium.clicked.connect(self.set_medium_model)
        self.newButton.clicked.connect(self.start_classification)
        self.stopButton.clicked.connect(self.stop_all)

        self.classifier_worker = None
        self.stt_worker = None
        self.model_size = "small"
        self.mode = None

        STT.setStyleSheet("""
            QWidget {
                border: 2px solid gray;
                border-radius: 10px;
                padding: 5px;
            }
        """)

        self.pushButton.setStyleSheet("""
            QPushButton {
                color: red;
                border: 2px solid red;
                border-radius: 10px;
                background-color: transparent;
                padding: 10px;
            }
            QPushButton:hover {
                background-color: #ffcccc;
            }
        """)

        self.pushButton_2.setStyleSheet("""
            QPushButton {
                color: blue;
                border: 2px solid blue;
                border-radius: 10px;
                background-color: transparent;
                padding: 10px;
            }
            QPushButton:hover {
                background-color: #cceeff;
            }
        """)

        self.newButton.setStyleSheet("""
            QPushButton {
                color: orange;
                border: 2px solid orange;
                border-radius: 10px;
                background-color: transparent;
                padding: 10px;
            }
            QPushButton:hover {
                background-color: #ffcc99;
            }
        """)

        self.pushButton_3.setStyleSheet("""
            QPushButton {
                color: gray;
                border: 2px solid gray;
                border-radius: 10px;
                background-color: transparent;
                padding: 10px;
            }
            QPushButton:hover {
                background-color: #e0e0e0;
            }
        """)

        self.pushButton_small.setStyleSheet("""
            QPushButton {
                color: black;
                border: 2px solid black;
                border-radius: 10px;
                background-color: transparent;
                padding: 10px;
            }
            QPushButton:hover {
                background-color: #e0e0e0;
            }
        """)

        self.pushButton_medium.setStyleSheet("""
            QPushButton {
                color: black;
                border: 2px solid black;
                border-radius: 10px;
                background-color: transparent;
                padding: 10px;
            }
            QPushButton:hover {
                background-color: #e0e0e0;
            }
        """)

        self.stopButton.setStyleSheet("""
            QPushButton {
                color: black;
                border: 2px solid black;
                border-radius: 10px;
                background-color: transparent;
                padding: 10px;
            }
            QPushButton:hover {
                background-color: #e0e0e0;
            }
        """)

        icon_path = os.path.join(os.getcwd(), 'resources', 'free-icon-font-play-3917517.png')
        if os.path.exists(icon_path):
            STT.setWindowIcon(QtGui.QIcon(icon_path))
        else:
            print(f"아이콘 경로를 찾을 수 없습니다: {icon_path}")

    def retranslateUi(self, STT):
        _translate = QtCore.QCoreApplication.translate
        STT.setWindowTitle(_translate("STT", "PLAY"))
        self.pushButton.setText(_translate("STT", "녹음"))
        self.pushButton_2.setText(_translate("STT", "종료"))
        self.pushButton_3.setText(_translate("STT", "출력 초기화"))
        self.pushButton_small.setText(_translate("STT", "small"))
        self.pushButton_medium.setText(_translate("STT", "medium"))
        self.newButton.setText(_translate("STT", "소리 분석"))
        self.stopButton.setText(_translate("STT", "모든 작업 중지"))


    
    def start_classification(self):
        if self.mode == "STT": 
            self.stop_recording()

        if self.classifier_worker is None or not self.classifier_worker.isRunning():
            self.textEdit.append("소리 분석을 시작합니다...")
            self.classifier_worker = SoundClassifierWorker()
            self.classifier_worker.sound_classified.connect(self.update_text)
            self.classifier_worker.start()
            self.mode = "classification"  

    def start_recording(self):
        if self.mode == "classification":  
            self.stop_classification()

        if self.stt_worker is None or not self.stt_worker.isRunning():
            self.textEdit.append(f"녹음을 시작합니다... (모델: {self.model_size})")
            self.stt_worker = STTWorker(model_size=self.model_size)
            self.stt_worker.text_update.connect(self.update_text)
            self.stt_worker.stt_finished.connect(self.update_text)
            self.stt_worker.start()
            self.mode = "STT" 

    def stop_recording(self):
        if self.stt_worker is not None and self.stt_worker.isRunning():
            self.stt_worker.stop()
            self.stt_worker = None
            self.mode = None  

    def stop_classification(self):
        if self.classifier_worker is not None:
            self.classifier_worker.stop()
            self.classifier_worker = None
            self.textEdit.append("소리 분석이 중지되었습니다.")
            self.mode = None  

    def stop_all(self):
        if self.mode == "STT":
            self.stop_recording()
        elif self.mode == "classification":
            self.stop_classification()

    def clear_text(self):
        self.textEdit.clear()

    def set_small_model(self):
        self.model_size = "small"
        self.textEdit.append("모델이 'small'로 설정되었습니다.")

    def set_medium_model(self):
        self.model_size = "medium"
        self.textEdit.append("모델이 'medium'로 설정되었습니다.")

    def update_text(self, text):
        self.textEdit.append(f"{text}")

if __name__ == "__main__":
    import sys
    app = QtWidgets.QApplication(sys.argv)
    STT = QtWidgets.QWidget()
    ui = Ui_STT()
    ui.setupUi(STT)
    STT.show()
    sys.exit(app.exec_())

c:\Anaconda\envs\home2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1/1 [==============================] - 0s 100ms/step
---------------------------------------------
1. dog_bark: 17.53% (2024-10-17 14:34:30)
2. siren: 14.51% (2024-10-17 14:34:30)
3. drilling: 1.37% (2024-10-17 14:34:30)
4. car_horn: 0.55% (2024-10-17 14:34:30)
5. jackhammer: 0.08% (2024-10-17 14:34:30)
---------------------------------------------
1/1 [==============================] - 0s 12ms/step
---------------------------------------------
1. dog_bark: 84.61% (2024-10-17 14:34:35)
2. car_horn: 4.33% (2024-10-17 14:34:35)
3. siren: 3.75% (2024-10-17 14:34:35)
4. drilling: 3.10% (2024-10-17 14:34:35)
5. jackhammer: 0.33% (2024-10-17 14:34:35)
---------------------------------------------
1/1 [==============================] - 0s 12ms/step
---------------------------------------------
1. siren: 52.21% (2024-10-17 14:34:40)
2. dog_bark: 20.47% (2024-10-17 14:34:40)
3. car_horn: 9.43% (2024-10-17 14:34:40)
4. drilling: 1.10% (2024-10-17 14:34:40)
5. jackhammer: 0.09% (2024-10-17 14:34:

SystemExit: 0

c:\Anaconda\envs\home2\lib\site-packages\IPython\core\interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


1/1 [==============================] - 0s 12ms/step
---------------------------------------------
1. siren: 41.58% (2024-10-17 14:35:22)
2. dog_bark: 28.49% (2024-10-17 14:35:22)
3. drilling: 0.13% (2024-10-17 14:35:22)
4. car_horn: 0.01% (2024-10-17 14:35:22)
5. jackhammer: 0.00% (2024-10-17 14:35:22)
---------------------------------------------
1/1 [==============================] - 0s 13ms/step
---------------------------------------------
1. siren: 17.08% (2024-10-17 14:35:27)
2. dog_bark: 5.76% (2024-10-17 14:35:27)
3. drilling: 0.17% (2024-10-17 14:35:27)
4. car_horn: 0.01% (2024-10-17 14:35:27)
5. jackhammer: 0.00% (2024-10-17 14:35:27)
---------------------------------------------
1/1 [==============================] - 0s 12ms/step
---------------------------------------------
1. siren: 20.12% (2024-10-17 14:35:32)
2. dog_bark: 5.90% (2024-10-17 14:35:32)
3. drilling: 0.55% (2024-10-17 14:35:32)
4. car_horn: 0.01% (2024-10-17 14:35:32)
5. jackhammer: 0.00% (2024-10-17 14:35:32

c:\Anaconda\envs\home2\lib\site-packages\librosa\core\spectrum.py:2944: RuntimeWarning: overflow encountered in square
  np.abs(


---------------------------------------------
1. car_horn: nan% (2024-10-17 15:58:58)
2. dog_bark: nan% (2024-10-17 15:58:58)
3. drilling: nan% (2024-10-17 15:58:58)
4. jackhammer: nan% (2024-10-17 15:58:58)
5. siren: nan% (2024-10-17 15:58:58)
---------------------------------------------
